In [ ]:
# we need to develop the map from yield to cash
#data from marketing study done (2014 Solenkiatal)
# 2,195.00 2,258.00 2,290.00 2,241.00 in rupees per quintal received by the farmer
# but also 69,500.00 in returns, with costs of 32,071.44, so net income around 37,428.56
returns = 66080.00
yields = 20.65 
revenue_per_quintal = returns / yields
fixed_cost = 6251.75
variable_cost = 25711.87 - 901.43 # removed family cost as its not a direct cash cost
total_cost_per_hectare = fixed_cost + variable_cost
total_cost_per_hectare, revenue_per_quintal # 1 quintal = 100 kg of crops

uniform_rainfall_numbers = np.arange(600,1500,5)
quad_unif_rain = quadratic.fit_transform(uniform_rainfall_numbers.reshape(-1,1))
predicted_yield = quadratic_model.predict(quad_unif_rain)
plt.scatter(uniform_rainfall_numbers, predicted_yield)
#now just run the money plot for these guys
rev_per_hectare = predicted_yield / 100 * revenue_per_quintal
profit_per_hectare = rev_per_hectare - total_cost_per_hectare
profit_per_hectare
beginning_index = -1
ending_index = -1
for i in range(len(profit_per_hectare)):
    if profit_per_hectare[i] > 0 and beginning_index == -1:
        beginning_index = i
    if profit_per_hectare[i] < 0 and beginning_index >= 0:
        ending_index = i
        break


profit_per_hectare[beginning_index], uniform_rainfall_numbers[beginning_index]
profit_per_hectare[ending_index], uniform_rainfall_numbers[ending_index]
low_thresh, up_thresh = uniform_rainfall_numbers[beginning_index], uniform_rainfall_numbers[ending_index]
plt.plot(profit_per_hectare)
plt.hlines(y=0, xmin = -10, xmax = 200)

#now create simulation by randomly drawing from rainfall distribution, per hectare
def simulate_no_policy(n_trials, n_years, seed=98765):
    rng = np.random.default_rng(seed)
    random_years = rng.normal(east_monsoon_mean, east_monsoon_std, size = (n_trials,n_years))
    profit_n_trials = []
    for _ in range(n_trials):
        quad_i = quadratic.fit_transform(random_years[_].reshape(-1,1))
        yield_i = quadratic_model.predict(quad_i)
        profit_i = revenue_per_quintal * ( yield_i / 100 ) - total_cost_per_hectare
        profit_n_trials.append(np.sum(profit_i))
    return profit_n_trials
no_policy_output = simulate_no_policy(100, 1)
np.mean(no_policy_output) # single hectare
# under these assumptions, insurance less always results in a loss in the long run 
plt.plot(no_policy_output)
#now try naive single payout, single lower threshold insurance policy simulation
def simulate_naive_simple(n_trials, n_years, lower_threshold, payout, seed=98765):
    rng = np.random.default_rng(seed)
    random_years = rng.normal(east_monsoon_mean, east_monsoon_std, size = (n_trials,n_years))
    profit_n_trials = []
    for _ in range(n_trials):
        rainfall_i = random_years[_]
        insurance_payouts_i = [payout if x < lower_threshold else 0 for x in rainfall_i]
        quad_i = quadratic.fit_transform(random_years[_].reshape(-1,1))
        yield_i = quadratic_model.predict(quad_i)
        profit_i = revenue_per_quintal * ( yield_i / 100 ) - total_cost_per_hectare + insurance_payouts_i
        profit_n_trials.append(np.sum(profit_i))
    return profit_n_trials
naive_simple_profits = simulate_naive_simple(100, 1, low_thresh, 400)
plt.plot(naive_simple_profits)
#now try naive single payout, double threshold insurance policy simulation
def simulate_double_threshold(n_trials, n_years, lower_threshold, upper_threshold, lower_payout, upper_payout, seed=98765):
    rng = np.random.default_rng(seed)
    random_years = rng.normal(east_monsoon_mean, east_monsoon_std, size = (n_trials,n_years))
    profit_n_trials = []
    rainfall_n_trials = []
    for _ in range(n_trials):
        rainfall_i = random_years[_]
        insurance_payouts_i = []
        for x in rainfall_i:
            if x < lower_threshold:
                insurance_payouts_i.append(lower_payout)
            elif x > upper_threshold:
                insurance_payouts_i.append(upper_payout)
            else:
                insurance_payouts_i.append(0)
        quad_i = quadratic.fit_transform(random_years[_].reshape(-1,1))
        yield_i = quadratic_model.predict(quad_i)
        rainfall_n_trials.append(yield_i)
        profit_i = revenue_per_quintal * ( yield_i / 100 ) - total_cost_per_hectare + insurance_payouts_i
        profit_n_trials.append(np.sum(profit_i))
    plt.plot(profit_n_trials)
    return profit_n_trials
double_threshold_profits = simulate_double_threshold(100, 1, low_thresh, up_thresh, 200, 200)
#insurance policy parameters, in expectation 
plt.plot(double_threshold_profits)
#now try naive linear payout, single lower threshold insurance policy simulation
def simulate_linear_payout(n_trials, n_years, lower_threshold, lower_payout, seed=98765):
    rng = np.random.default_rng(seed)
    random_years = rng.normal(east_monsoon_mean, east_monsoon_std, size = (n_trials,n_years))
    profit_n_trials = []
    for _ in range(n_trials):
        rainfall_i = random_years[_]
        insurance_payouts_i = []
        for x in rainfall_i:
            if x < lower_threshold:
                insurance_payouts_i.append(lower_payout * (lower_threshold - x))
            else:
                insurance_payouts_i.append(0)
        quad_i = quadratic.fit_transform(random_years[_].reshape(-1,1))
        yield_i = quadratic_model.predict(quad_i)
        profit_i = revenue_per_quintal * ( yield_i / 100 ) - total_cost_per_hectare + insurance_payouts_i
        profit_n_trials.append(np.sum(profit_i))
    plt.plot(profit_n_trials)
    return profit_n_trials
linear_profits = simulate_linear_payout(100, 1, low_thresh, 10)